In [3]:
print("hellow world")

import requests

hellow world


In [4]:
url = "http://host.docker.internal:11500/api/generate"
payload = {
    "model": "qwen3:4b",       # замените на нужную модель
    "prompt": "Напиши очень при очень краткое резюме по теме 'формальная верификация в coq' на русском языке",
    "stream": False
}

In [11]:
resp = requests.post(url, json=payload, timeout=1200)

In [12]:
resp.raise_for_status()
data = resp.json()  # вернёт JSON с результатом
print(data)

{'model': 'qwen3:4b', 'created_at': '2025-08-27T10:48:26.9258729Z', 'response': '<think>\nХорошо, мне нужно написать очень краткое резюме по теме "формальная верификация в Coq" на русском языке. Сначала я должен понять, что такое формальная верификация в Coq. Coq — это системный инструмент для формальной верификации программ и теорем. Он используется в математике и программировании для доказательства теорем и проверки правильности программ.\n\nФормальная верификация в Coq означает, что программу или теорему можно проверять строго математически, используя логику и формальные методы. Coq использует типы и логику, чтобы доказывать теоремы и убедиться, что код корректен.\n\nНужно кратко объяснить, что Coq — это систем для формальной верификации, где разработчики доказывают теоремы и проверяют корректность программ через формальные методы. Важно упомянуть, что это используется в безопасном программировании, криптографии и других областях.\n\nВажно уложиться в очень краткое резюме, так что н

In [22]:
import requests
import json
from typing import Optional

API_BASE = "http://host.docker.internal:11500"  # Базовый URL API Ollama
GENERATE_PATH = "/api/generate"  # Путь для генерации текста

# Кастомное исключение для ошибок работы с Ollama API
class OllamaAPIError(Exception):
    pass

def check_server() -> None:
    """
    Проверяет доступность сервера Ollama.
    Бросает исключение, если сервер недоступен или ответ не соответствует ожиданиям.
    """
    try:
        r = requests.get(API_BASE + "/", timeout=5)  # Запрос к корневому пути
    except requests.RequestException as e:
        # Если соединение не удалось
        raise OllamaAPIError(f"Cannot reach {API_BASE}: {e}")
    # Проверка статуса и содержимого ответа
    if r.status_code != 200 or "Ollama" not in (r.text or ""):
        raise OllamaAPIError(f"Service at {API_BASE} responded unexpectedly: status={r.status_code}, body={r.text!r}")

def extract_text_from_json(obj) -> Optional[str]:
    """
    Извлекает текст из JSON-объекта, который может содержать ответ модели.
    Поддерживает разные форматы (OpenAI-like, Ollama-like).
    """
    if obj is None:
        return None
    if isinstance(obj, str):
        return obj
    if isinstance(obj, dict):
        # Проверка распространённых ключей
        for key in ("response", "text", "completion", "content"):
            if key in obj and isinstance(obj[key], str):
                return obj[key]
        # Обработка формата с choices[]
        ch = obj.get("choices")
        if isinstance(ch, list) and ch:
            first = ch[0]
            if isinstance(first, dict):
                # OpenAI streaming формат: delta.content
                if "delta" in first and isinstance(first["delta"], dict):
                    return first["delta"].get("content")
                # Прямой текст
                if "text" in first and isinstance(first["text"], str):
                    return first["text"]
                # Формат message.content
                if "message" in first and isinstance(first["message"], dict):
                    return first["message"].get("content")
    return None

def stream_until_dot(goal_text: str, f, failed_tactics = [], model: str = "qwen3:8b", timeout: int = 120) -> str:
    """
    Отправляет запрос к Ollama API с целью сгенерировать одну строку тактики Coq.
    Читает потоковый ответ до первой точки '.' и возвращает собранный текст.
    
    :param goal_text: Цель в синтаксисе Coq, для которой требуется тактика
    :param model: Название модели (по умолчанию llama3:8b)
    :param timeout: Таймаут запроса в секундах
    :return: Сгенерированная тактика (строка, заканчивающаяся точкой)
    """
    check_server()  # Проверка доступности сервера

    url = API_BASE + GENERATE_PATH
    # Формируем инструкцию для модели
    failed_tactics = '\n'.join(failed_tactics)
    prompt = f"""You are a Coq tactic suggester.

INPUT:
A Coq goal is provided below in its ORIGINAL Coq format. Do not modify or repeat it.
A list of previously tried tactics that FAILED is also provided; avoid using them again.

GOAL:
```coq
{goal_text}
````

FAILED TACTICS:
{failed_tactics}

TASK:
Output EXACTLY ONE Coq tactic line that makes the most direct progress toward solving the goal and is NOT in the failed tactics list.

HARD RULES:

* Output ONLY a single Coq tactic line terminated by a period.
* No explanations, no prefixes, no code fences, no extra text, no alternatives.
* Do NOT repeat or alter the goal.
* Prefer the most direct, type-correct next step.
* If multiple options exist, choose the single most straightforward one.
* Semicolons (`;`) and proof brackets (`[...]`) are allowed, but the result must be ONE line.
* Never output a tactic that appears in the FAILED TACTICS list.
* Never answer "I don't know" or similar; always output your best single tactic.
* Keep it minimal.
* At the end it is necessary to have `.`

Respond with ONLY the tactic line. """
    # Тело запроса для Ollama API
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": True  # Включаем потоковый режим
    }
    
    # Отправка запроса
    resp = requests.post(url, json=payload, stream=True, timeout=timeout)
    if resp.status_code == 404:
        # Ошибка 404 — сервер не настроен или не запущен
        resp.close()
        raise OllamaAPIError(
            "404 from /api/generate. Проверьте: контейнер запущен ли с 'ollama serve', доступен ли порт 11434, и не занят ли порт другим процессом. "
            "Выполните `curl http://localhost:11434/` и `docker logs ollama`."
        )
    resp.raise_for_status()  # Проверка на другие HTTP-ошибки
    
    collected = ""  # Буфер для символов
    is_collect = bool(model == "qwen:4b")
    try:
        # Читаем поток построчно
        for raw_line in resp.iter_lines(decode_unicode=True):
            if raw_line is None:
                continue
            line = raw_line.strip()
            if not line:
                continue
            # Попытка распарсить строку как JSON
            try:
                obj = json.loads(line)
            except json.JSONDecodeError:
                # Если не JSON — используем как текст
                text = line
            else:
                # Извлекаем полезный текст из объекта
                text = extract_text_from_json(obj) or ""
            if not text:
                continue

            print(text, end="", file=f)

            if "</think>" in text:
                is_collect = True
                text = text.split("</think>")[-1]
            
            # Ищем первую точку в ответе
            if is_collect:
                for ch in text:
                    if ch == ".":
                        # Закрываем соединение и возвращаем собранный ответ
                        resp.close()
                        return collected.strip() + "."
                    collected += ch
        # Если поток закончился без точки — возвращаем что есть
        resp.close()
        return collected.strip()
    finally:
        # Гарантированное закрытие соединения
        try:
            resp.close()
        except Exception:
            pass


In [12]:
for i in range(0):
        print (stream_until_dot(
    """1 goal
l : list nat
Hall : forall x : nat, In x l
______________________________________(1/1)
In (S (max_list l)) l
    """), "\n______________________________________\n\n\n")

In [13]:
!pip install sexpdata


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [26]:
!pip install PyTorch

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-m050fkri/pytorch_f564d9a29e944bbcac22ae029fc62818/setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for PyTorch
  Running setup.py clean for PyTorch
Failed to build PyTorch
ERROR: Could not build wheels for PyTorch, which is required to install pyproject.toml-based projects

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [23]:
import subprocess, sexpdata
from sexpdata import loads
import select

def read(proc):
    timeout = 5  # секунд
    ready, _, _ = select.select([proc.stdout], [], [], timeout)
    if not ready:
        return "eror"
    
    line = "("
    while "Completed" not in line:
        line += proc.stdout.readline()
        #print(line.split("\n")[-2])
    return line + ")"

def Add(proc, line):
    cmd = f'(Add () "{line}")\n'
    proc.stdin.write(cmd)
    proc.stdin.flush()
    return read(proc)

def Exec(proc, i):
    cmd = f'(Exec {i})\n'
    proc.stdin.write(cmd)
    proc.stdin.flush()
    return read(proc)

def Cancel(proc, i):
    cmd = f'(Cancel ({i}))\n'
    proc.stdin.write(cmd)
    proc.stdin.flush()
    return read(proc)

def Query(proc, i):
    cmd = f'(Query ((sid {i}) (pp ((pp_format PpStr)))) Goals)\n'
    proc.stdin.write(cmd)
    proc.stdin.flush()
    return read(proc)

def find_a_leters(tree, leters, path="0"):
    results = []
    if isinstance(tree, list) or isinstance(tree, tuple):
        if tree and isinstance(tree[0], sexpdata.Symbol) and tree[0].value() in leters:
            cur_tree = []
            for i, elem in enumerate(tree):
                if i != 0 and elem and isinstance(elem, list) \
                    and isinstance(elem[0], sexpdata.Symbol) and elem[0].value() in leters:
                    break
                cur_tree.append(elem)
            results.append((path, cur_tree))
        for i, elem in enumerate(tree):
            results += find_a_leters(elem, leters, path + "." + str(i))
    return results

def solve(lemma):
    proc = subprocess.Popen(
        ["sertop"],
        stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    line = proc.stdout.readline()
    while "(contents Processed)" not in line:
        line = proc.stdout.readline()

    added = 2
    Add(proc, str(lemma))
    Exec(proc, added)
    ans = find_a_leters(loads(Query(proc, added)), "CoqString")[0][1][1]

    with open('out.txt', 'a') as f:
        print(ans, file=f)
        print("============================", file=f)
    print(ans)
    print("============================")

    failed_tactics = []

    cmd = f'(Query ((sid {added})) Goals)\n'
    proc.stdin.write(cmd)
    proc.stdin.flush()
    count = len(find_a_leters(loads(read(proc)), "info"))

    is_ai = False

    while count:
        tactic = "some eror.."
        if is_ai:
            with open('out.txt', 'a') as f:
                tactic = stream_until_dot(ans, f, failed_tactics)
        else:
            tactic = input()
            if tactic == "ai": 
                is_ai = True
                continue
            if tactic == "er":
                return
        #tactic = "some eror.."
        
        if "." != tactic[-1] : tactic += '.'
        with open('out.txt', 'a') as f:
            print("\n<<<", tactic, file=f)
        print("<<", tactic)
        Add(proc, tactic)
        added += 1
        ex = find_a_leters(loads(Exec(proc, added)), "CoqExn")

        if not ex:
            ans_new = find_a_leters(loads(Query(proc, added)), "CoqString")[0][1][1]

            if ans_new == ans:
               ex = loads('(str"tactics do not change the goal")')
            
        
        if not ex:           
            cmd = f'(Query ((sid {added})) Goals)\n'
            proc.stdin.write(cmd)
            proc.stdin.flush()
            new_count = len(find_a_leters(loads(read(proc)), "info"))
            
            #print("count", count)
            #print("new_count", new_count)
            if new_count == 0:
                Add(proc, "Qed.")
                added += 1
                ex = find_a_leters(loads(Exec(proc, added)), "CoqExn")
                if ex:
                    Cancel(proc, added)
                    print("<< ", end="")
                    ex = []
                    while not ex:
                        Add(proc, "}")
                        added += 1
                        ex = find_a_leters(loads(Exec(proc, added)), "CoqExn")
                        if not ex:
                            print("} ", end="")

                    Cancel(proc, added)
                    Add(proc, "Qed.")
                    added += 1
                    
                ex = find_a_leters(loads(Exec(proc, added)), "CoqExn")
                if ex:
                    print("error")
                print("\nQed.")
                return
            if new_count < count:
                Add(proc, "}")
                print("<< } ", end="")
                added += 1
                ex = find_a_leters(loads(Exec(proc, added)), "CoqExn")
                while not ex:
                    Add(proc, "}")
                    added += 1
                    ex = find_a_leters(loads(Exec(proc, added)), "CoqExn")
                    if not ex:
                        print("} ", end="")
                Cancel(proc, added)
                Add(proc, "{")
                added += 1
                print("{")
            elif new_count > count:
                Add(proc, "{")
                print("<< {")
                added += 1

            count = new_count
            
            failed_tactics = []

            Exec(proc, added)
            ans = find_a_leters(loads(Query(proc, added)), "CoqString")[0][1][1]
            with open('out.txt', 'a') as f:
                print(ans, file=f)
                print("============================", file=f)
            print(ans)
            print("============================")
        else:
            with open('out.txt', 'a') as f:
                print("eror", file=f)
                print("============================", file=f)
            print("eror")
            print("============================")
            Cancel(proc, added)
            
            failed_tactics.append(str((tactic, find_a_leters(ex, "str")[0][1][1])))
            with open('out.txt', 'a') as f:
                print(failed_tactics[-1], file=f)
                print("============================", file=f)
            print(failed_tactics[-1])
            print("============================")
    

In [24]:
# solve("Lemma test: forall (x y z : nat), x = y-> y = z -> x = z.")
# solve("Lemma test: forall (x y : nat), x + S y = S (x + y).")
# solve("Lemma test: forall (x : nat), x + 0 = x.")
solve("Lemma test: forall (x y : nat), x + y = y + x.")
# solve("Goal forall A B C : Prop, (A -> (B -> C)) -> ((A -> B) -> (A -> C)).")
# solve("Goal forall A B C D: Prop, (A -> C)/\(B -> D) -> A/\B -> C/\D.")
# solve("Goal forall A : Prop, A -> ~ ~ A.")

none
forall x y : nat, x + y = y + x


 ai


KeyboardInterrupt: 

In [79]:
proc = subprocess.Popen(
    ["sertop"],
    stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
)
line = proc.stdout.readline()
while "(contents Processed)" not in line:
    line = proc.stdout.readline()

printAdd(proc, "Lemma test: forall (x y : nat), x = y -> y = x.")

((Answer 0 Ack)
(Answer 0(Added 2((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 47))NewAddTip))
(Answer 0 Completed)
)


In [80]:
print(Add(proc, "Proof."))

((Answer 1 Ack)
(Answer 1(Added 3((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 6))NewAddTip))
(Answer 1 Completed)
)


In [81]:
print(Add(proc, "intros x y H."))

((Answer 2 Ack)
(Answer 2(Added 4((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 13))NewAddTip))
(Answer 2 Completed)
)


In [83]:
print(Add(proc, "rewrite H."))

((Answer 4 Ack)
(Answer 4(Added 6((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 10))NewAddTip))
(Answer 4 Completed)
)


In [5]:
Add(proc, "auto.")

'((Answer 2 Ack)\n(Answer 2(Added 5((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 5))NewAddTip))\n(Answer 2 Completed)\n)'

In [45]:
print(Cancel(proc, 5))

((Answer 16 Ack)
(Feedback((doc_id 0)(span_id 4)(route 0)(contents Processed)))
(Answer 16(Canceled(5)))
(Answer 16 Completed)
)


In [84]:
for i in range(1, 7):
    print(Exec(proc, i), "\n")

((Answer 5 Ack)
(Feedback((doc_id 0)(span_id 1)(route 0)(contents Processed)))
(Answer 5 Completed)
) 

((Answer 6 Ack)
(Feedback((doc_id 0)(span_id 2)(route 0)(contents(ProcessingIn master))))
(Feedback((doc_id 0)(span_id 1)(route 0)(contents Processed)))
(Feedback((doc_id 0)(span_id 2)(route 0)(contents Processed)))
(Answer 6 Completed)
) 

((Answer 7 Ack)
(Feedback((doc_id 0)(span_id 3)(route 0)(contents(ProcessingIn master))))
(Feedback((doc_id 0)(span_id 2)(route 0)(contents Processed)))
(Feedback((doc_id 0)(span_id 3)(route 0)(contents Processed)))
(Answer 7 Completed)
) 

((Answer 8 Ack)
(Feedback((doc_id 0)(span_id 4)(route 0)(contents(ProcessingIn master))))
(Feedback((doc_id 0)(span_id 3)(route 0)(contents Processed)))
(Feedback((doc_id 0)(span_id 4)(route 0)(contents Processed)))
(Answer 8 Completed)
) 

((Answer 9 Ack)
(Feedback((doc_id 0)(span_id 5)(route 0)(contents(ProcessingIn master))))
(Feedback((doc_id 0)(span_id 4)(route 0)(contents Processed)))
(Feedback((doc_id 0)

In [85]:
for i in range(1, 7):
    print(Query(proc, i), "\n")

((Answer 11 Ack)
(Answer 11(ObjList()))
(Answer 11 Completed)
) 

((Answer 12 Ack)
(Answer 12(ObjList((CoqString"none\n============================\nforall x y : nat, x = y -> y = x"))))
(Answer 12 Completed)
) 

((Answer 13 Ack)
(Answer 13(ObjList((CoqString"none\n============================\nforall x y : nat, x = y -> y = x"))))
(Answer 13 Completed)
) 

((Answer 14 Ack)
(Answer 14(ObjList((CoqString"\n  H : x = y\n  x, y : nat\n============================\ny = x"))))
(Answer 14 Completed)
) 

((Answer 15 Ack)
(Answer 15(ObjList((CoqString"\n  H : x = y\n  x, y : nat\n============================\ny = y"))))
(Answer 15 Completed)
) 

((Answer 16 Ack)
(Answer 16(ObjList()))
(Answer 16 Completed)
) 



In [69]:
i = 1
while "eror" not in Exec(i) and "CoqExn" not in Exec(i):
    i += 1
    print(i, end="")

234